In [96]:
import pandas as pd 
import seaborn as sns
sns.set(style='ticks')
%matplotlib inline

In [19]:
ls /Data/common/

height.csv           stats.csv
idealBodyWeight.csv  tv_subset_with_set_volume.csv
RData/               tv_subset_with_set_volume_merged_w_idealbodyWeight.csv


In [239]:
height =  pd.read_csv('/Data/common/height.csv')
height = height[~height.icustay_id.isnull()].set_index('icustay_id')
stats = pd.read_csv('/Data/common/stats.csv')
stats = stats[~stats.icustay_id.isnull()].set_index('icustay_id')
body = pd.read_csv('/Data/common/idealBodyWeight.csv')
body = body[~body.icustay_id.isnull()].set_index('icustay_id')

In [184]:
subset = pd.read_csv('/Data/common/tv_subset_with_set_volume_merged_w_idealbodyWeight.csv')
subset = subset[~subset.icustay_id.isnull()].set_index('icustay_id')
subset = subset[subset.tv_pbw > 0]

In [185]:
subset.shape

(16865, 7)

In [255]:
ids = set(subset.index).intersection(set(body.index)).intersection(set(height.index)).intersection(stats.index)

dfList = [subset, stats[[i for i in stats.columns if i not in ['ethnicity', 'gender']]]]

dfList = [i.loc[ids,:] for i in dfList]

df_final = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='left'), dfList)

In [269]:
cleaned = df_final.loc[:,sorted([i for i in df_final.columns if  ('Un' not in i)])]

cleaned.columns

In [316]:
demo = cleaned.reset_index()[['icustay_id', 'gender', 'age', 'height', 'subject_id', 
                              'hadm_id', 'admission_type', 'admittime', 'dischtime', 'ethnicity',
       'first_hosp_stay', 'first_icu_stay',   
       'hospital_expire_flag', 'hospstay_seq', 'icustay_seq', 'intime',
       'los_hospital', 'los_icu', 'outtime']]

demo.to_csv('Ventilator_demo.tsv', sep='\t', index=False)

In [278]:
vent = cleaned.reset_index()[['icustay_id', 'height', 'pbw','tv_set' ,'tv_pbw' ]]

vent.to_csv('Ventilator_vent.tsv',sep='\t', index=False)

In [281]:
!rsync -aP Ventilator_* /Data/common/

sending incremental file list
Ventilator_big_table.tsv
      1,311,749 100%  152.47MB/s    0:00:00 (xfr#1, to-chk=2/3)
Ventilator_demo.tsv
      2,716,731 100%  107.95MB/s    0:00:00 (xfr#2, to-chk=1/3)
Ventilator_vent.tsv
        790,117 100%   26.91MB/s    0:00:00 (xfr#3, to-chk=0/3)


In [259]:
cleaned.shape

(16865, 16)

In [234]:
cleaned.reset_index().to_csv('Ventilator_big_table.tsv', sep='\t', index=False)

In [340]:
apache = pd.read_csv('/Data/common/apache.csv')
oasis =  pd.read_csv('/Data/common/oasis.csv')

apache = demo.merge(apache[[i for i in apache.columns if i not in apache_cols[:-1]]], on='icustay_id')

apache_cols = list(set(apache.columns).intersection(set(demo.columns)))

oasis_cols = list(set(apache.columns).intersection(set(oasis.columns)))

In [343]:
with_apa_oas = apache.merge(oasis[[i for i in oasis.columns if i not in oasis_cols[::2]]], on='icustay_id')

big_with_opa = with_apa_oas.drop('Unnamed: 0', axis=1)

big_with_opa.shape

In [357]:
mort = pd.read_csv('/Data/common/mort_sum_full.csv', encoding='utf8')

big_with_morty = big_with_opa.merge(mort[[i for i in mort.columns if i not in ['hadm_id','subject_id']]], on='icustay_id')

In [476]:
mort.icu_mort.unique()

array([  0.,   1.,  nan])

In [364]:
big_with_morty.shape

(16865, 28)

In [365]:
big_with_morty.head()

,icustay_id,gender,age,height,subject_id,hadm_id,admission_type,admittime,dischtime,ethnicity,...,outtime,apsiii,apsiii_prob,oasis,oasis_prob,icu_mort,h_mort,one_mort,thr_mort,six_mort
0,294912,M,57.0272,178.00,72530,189421,EMERGENCY,2170-07-11 17:28:00,2170-07-30 16:33:00,WHITE,...,2170-07-17 14:33:28,31,0.048756,38,0.209226,0.0,0,0.0,0.0,0.0
1,262146,F,77.9385,157.48,14114,189388,ELECTIVE,2141-04-25 10:00:00,2141-05-04 17:00:00,WHITE,...,2141-04-27 12:52:34,41,0.075975,37,0.188911,0.0,0,0.0,0.0,0.0
2,229381,F,42.2699,165.10,47927,161682,EMERGENCY,2108-12-23 17:42:00,2108-12-27 19:00:00,UNKNOWN/NOT SPECIFIED,...,2108-12-27 21:00:12,74,0.281151,48,0.486353,0.0,1,1.0,1.0,1.0
3,294919,M,70.8541,172.72,86078,182281,EMERGENCY,2183-02-03 21:02:00,2183-02-08 15:00:00,WHITE,...,2183-02-08 17:25:51,33,0.053332,34,0.137099,0.0,1,1.0,1.0,1.0
4,262154,F,300.0023,157.00,94698,152925,EMERGENCY,2105-12-07 10:37:00,2105-12-26 15:30:00,WHITE,...,2105-12-19 18:35:54,81,0.352529,57,0.748927,0.0,0,0.0,0.0,0.0


In [366]:
big_with_morty.to_csv('/Data/common/Ventilator_big_table_oasis_apache_morbidity.csv', sep=',', index=False)

In [368]:
big_with_morty.columns

Index(['icustay_id', 'gender', 'age', 'height', 'subject_id', 'hadm_id',
       'admission_type', 'admittime', 'dischtime', 'ethnicity',
       'first_hosp_stay', 'first_icu_stay', 'hospital_expire_flag',
       'hospstay_seq', 'icustay_seq', 'intime', 'los_hospital', 'los_icu',
       'outtime', 'apsiii', 'apsiii_prob', 'oasis', 'oasis_prob', 'icu_mort',
       'h_mort', 'one_mort', 'thr_mort', 'six_mort'],
      dtype='object')

In [478]:
ls /Data/common

apache.csv           RData/
first_icu_id.csv     stats.csv
firstVisits.csv      time_till_death.csv
forAlvin.csv         tvObserved_filtered.csv
for_kiesha/          tvSet_filtered.csv
height.csv           tv_subset_with_set_volume.csv
idealBodyWeight.csv  tv_subset_with_set_volume_merged_w_idealbodyWeight.csv
meanNmax.csv         Ventilator_big_table_oasis_apache_morbidity.csv
mort_icu.csv         Ventilator_big_table.tsv
mort_sum.csv         Ventilator_vent.tsv
mort_sum_full.csv    ventmodes_metavision.csv
oasis.csv            ventmodes_metavision_inclusion_annotated.csv
raw_mort.csv         ventmodes_metavision_inclusion_annotated.csv~


In [384]:
tvSet = pd.read_csv('/Data/common/tvSet_filtered.csv')

tvOb = pd.read_csv('/Data/common/tvObserved_filtered.csv')

tvOb_fil = tvOb[['icustay_id','tv_pbw']]
tvOb_fil.columns = ['icustay_id','tv_pbw_observed']

In [395]:
set(tvSet.columns).intersection(set(tvOb.columns))

{'ethnicity', 'gender', 'height', 'icustay_id', 'pbw', 'tv_pbw', 'tv_set'}

In [408]:
## patients with tvObs but not tvSet
missing = set(tvOb_fil.icustay_id).difference(set(tvSet.icustay_id))

vent_data = tvSet.merge(tvOb_fil, on='icustay_id')

final_big_table = big_with_morty[[i for i in big_with_morty.columns if i not in ['ethnicity', 'gender', 'height']]].merge(vent_data, on='icustay_id')

final_big_table.to_csv('/Data/common/for_kiesha/Ventilator_final_table_demo_mort_vent.csv',',', index=False)

In [479]:
meanMax = pd.read_csv('/Data/common/meanNmax.csv')

In [ ]:
final_big_table

In [481]:
set(final_big_table.columns).intersection(set(meanMax))

{'ethnicity', 'gender', 'height', 'icustay_id', 'pbw'}

In [488]:
final_final_table = final_big_table[[ i for i in final_big_table.columns if i not in ['ethnicity', 'gender', 'height', 'pbw']]].merge(meanMax, on='icustay_id')

In [504]:
final_final_table.to_csv('/Data/common/for_kiesha/Ventilator_final_final_table_demo_mort_vent.csv', index=False)

In [491]:
ls -lahrt /Data/common/

total 41M
drwxr-xr-x 7 root        root        4.0K Jun 30 01:46 ../
drwxrwsr-x 2 team12user6 team12      4.0K Jul  1 05:14 RData/
-rw-rw-r-- 1 team12user7 team12      1.1M Jul  1 07:29 height.csv
-rw-rw-r-- 1 team12user7 team12       10M Jul  1 07:30 stats.csv
-rw-rw-r-- 1 team12user7 team12      1.5M Jul  1 07:30 idealBodyWeight.csv
-rw-rw-r-- 1 team12user7 team12      429K Jul  1 07:31 tv_subset_with_set_volume.csv
-rw-rw-r-- 1 team12user7 team12      1.1M Jul  1 08:07 tv_subset_with_set_volume_merged_w_idealbodyWeight.csv
-rw-r--r-- 1 team12user4 team12user4 1.3M Jul  1 09:17 Ventilator_big_table.tsv
-rw-r--r-- 1 team12user4 team12user4 772K Jul  1 09:30 Ventilator_vent.tsv
-rw-r--r-- 1 team12user1 team12      7.2M Jul  1 10:18 mort_icu.csv
-rw-rw-r-- 1 team12user7 team12      2.9M Jul  1 10:26 apache.csv
-rw-r--r-- 1 team12user1 team12      1.4M Jul  1 10:32 mort_sum.csv
-rw-r--r-- 1 team12user1 team12      898K Jul  1 10:40 first_icu_id.csv
-rw-rw-r-- 1 team12user7 team12      2.

In [428]:
compare = big_with_morty.loc[big_with_morty.icustay_id.isin(vent_data.icustay_id),
                   ['ethnicity', 'gender', 'height','icustay_id']].merge(vent_data[['ethnicity', 'gender', 'height','icustay_id']], on='icustay_id')

In [451]:
t1 = compare.ix[:,:3]
t2 = compare.ix[:,4:8]
t2.columns = t1.columns

In [500]:
temp = pd.read_csv('/Data/common/for_kiesha/data_for_survival_temp.csv')

In [503]:
temp.columns

Index(['Unnamed: 0', 'subject_id', 'icustay_id', 'age', 'hadm_id.x',
       'admission_type', 'admittime', 'dischtime', 'first_hosp_stay',
       'first_icu_stay', 'hospital_expire_flag', 'hospstay_seq', 'icustay_seq',
       'intime.x', 'los_hospital', 'los_icu', 'outtime', 'apsiii',
       'apsiii_prob', 'oasis', 'oasis_prob', 'icu_mort', 'h_mort', 'one_mort',
       'thr_mort', 'six_mort', 'tv_set', 'gender', 'ethnicity', 'height',
       'pbw', 'tv_pbw', 'tv_pbw_observed', 'tv_cat', 'intime.y', 'deathtime',
       'time_till_death', 'icutime', 'morttime', 'tmins', 'tdays', 'hadm_id.y',
       'congestive_heart_failure', 'cardiac_arrhythmias', 'valvular_disease',
       'pulmonary_circulation', 'peripheral_vascular', 'hypertension',
       'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumo

In [452]:
t1.head()

,ethnicity_x,gender_x,height_x
0,WHITE,M,172.72
1,WHITE,F,157.00
2,WHITE,M,180.00
3,WHITE,M,183.00
4,WHITE,M,182.88


In [453]:
t2.head()

,ethnicity_x,gender_x,height_x
0,WHITE,M,172.72
1,WHITE,F,157.00
2,WHITE,M,180.00
3,WHITE,M,183.00
4,WHITE,M,182.88


In [455]:
t1[(t1 != t2).any(1)]

,ethnicity_x,gender_x,height_x


In [435]:
compare.ix[:,:2].equals(compare.ix[:,2:4])

False

In [409]:
set(big_with_morty.columns).intersection(set(vent_data.columns))

{'ethnicity', 'gender', 'height', 'icustay_id'}

In [496]:
oxy = pd.read_csv('/Data/common/oxyparam.csv', sep=',')

In [497]:
oxy.head()

,icustay_id,charttime,fio2,o2flow,lactate,tidalvolume,ventilationrate,ventilator,intubated
0,200001,2181-11-25 19:27:00,NaN,NaN,1.4,NaN,NaN,NaN,NaN
1,200001,2181-11-26 11:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200001,2181-11-26 17:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200001,2181-11-26 18:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200001,2181-11-27 01:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [507]:
oxy_fil = oxy.ix[oxy.icustay_id.isin(demo.icustay_id.values.tolist()),:]

In [519]:
keep = oxy_fil.ix[:, 2:].dropna(how='all')

oxyCleaned  = oxy_fil[oxy_fil.index.isin(keep.index)]

oxyCleaned  = oxy_fil[oxy_fil.index.isin(keep.index)]

In [522]:
pf = pd.read_csv('/Data/common/for_kiesha/PF.csv')

In [534]:
pf.head()

,Unnamed: 0,icustay_id,charttime,specimen_pred,pao2fio2,peep
0,1,200001,2181-11-25 19:27:00,ART,NaN,NaN
1,2,200001,2181-11-26 11:07:00,ART,NaN,NaN
2,3,200001,2181-11-26 17:44:00,ART,NaN,NaN
3,4,200001,2181-11-26 18:56:00,ART,NaN,NaN
4,5,200001,2181-11-27 01:26:00,ART,NaN,NaN


In [538]:
pfList = [mean_pf.ix[:,1:] , max_pf.ix[:,1:], sd_pf.ix[:,1:]]

In [539]:
pDf = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='left'), pfList)

In [544]:
pFinal = pDf.ix[final_final_table.icustay_id.values.tolist(),:]

In [546]:
pFinal.columns = ['pao2fio2_mean','peep_mean','pao2fio2_max','peep_max','pao2fio2_sd', 'peep_sd']

In [548]:
pFinal.reset_index().to_csv('/Data/common/for_kiesha/PF.final.cleaned.csv', index=False)

In [528]:
max_pf = pf.groupby('icustay_id').max()

In [530]:
sd_pf = pf.groupby('icustay_id').std()

In [535]:
mean_pf = pf.groupby('icustay_id').mean()

In [ ]:
df.apply(lambda x:)

In [551]:
oxy_fil.ventilationrate.unique()

array([ nan])

In [521]:
oxyCleaned.groupby('icustay_id').mean()

,fio2,o2flow,lactate,tidalvolume,ventilationrate,ventilator,intubated
icustay_id,,,,,,,
200003,NaN,NaN,2.600000,NaN,NaN,NaN,NaN
200009,NaN,NaN,2.000000,NaN,NaN,NaN,NaN
200014,36.000000,NaN,0.950000,646.666667,NaN,NaN,NaN
200028,50.000000,NaN,1.200000,NaN,NaN,NaN,NaN
200029,100.000000,NaN,12.300000,NaN,NaN,NaN,NaN
200034,NaN,NaN,3.566667,NaN,NaN,NaN,NaN
200045,44.000000,NaN,2.480000,428.000000,NaN,NaN,NaN
200053,50.000000,NaN,1.300000,455.000000,NaN,NaN,NaN
200062,NaN,NaN,2.000000,NaN,NaN,NaN,NaN


In [315]:
for i in apache.columns:
    if ('_x' in i) | ('_y' in i):
        print(i)

gender_x
age_x
height_x
admission_type_x
admittime_x
dischtime_x
ethnicity_x
first_hosp_stay_x
first_icu_stay_x
hospital_expire_flag_x
hospstay_seq_x
icustay_seq_x
intime_x
los_hospital_x
los_icu_x
outtime_x
gender_y
age_y
height_y
subject_id_x
hadm_id_x
admission_type_y
admittime_y
dischtime_y
ethnicity_y
first_hosp_stay_y
first_icu_stay_y
hospital_expire_flag_y
hospstay_seq_y
icustay_seq_y
intime_y
los_hospital_y
los_icu_y
outtime_y
subject_id_y
hadm_id_y
